In [0]:
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline 
import matplotlib.pyplot as plt 

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [0]:
pca_num=12

In [0]:
import requests
import io
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import matplotlib.pyplot as plt

def retrieve_data(urls):
  data = []
  for url in urls:
    response = requests.get(url)
    response.raise_for_status()
    data.append(np.load(io.BytesIO(response.content)))
  return data

images, labels = retrieve_data([
    "https://www.dropbox.com/s/zilveq4b7f1fdpz/images.npy?dl=1",
    "https://www.dropbox.com/s/6oue6rbyfcsatxg/labels.npy?dl=1"
])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels)

## min-max scaling

In [0]:
scaler = StandardScaler()
scaler.fit(X_train.reshape(len(X_train), -1, ))
X_train=scaler.transform(X_train.reshape(len(X_train), -1, ))

## PCA 

In [7]:
pca = PCA(n_components=pca_num)
pca.fit(X_train.reshape(len(X_train), -1, ))

PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

> # Start **KFold**

In [0]:
folds = StratifiedKFold(n_splits=6, shuffle=True, random_state=1)

In [0]:
y = y_train

In [0]:
def do_train():
    oof_preds = np.zeros((len(X_train), ))
    preds = None

    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        train_objects = X_train[trn_]
        valid_objects = X_train[val_]
        y_train = y[trn_]
        y_valid = y[val_]
        print('train len', len(train_objects))
        print('valid len', len(valid_objects))
        
        trainX_pca = pca.transform(train_objects.reshape(len(train_objects), -1, ))
        validX_pca = pca.transform(valid_objects.reshape(len(valid_objects), -1, ))

        model = RandomForestClassifier(n_estimators=500, max_depth=30, verbose=3, n_jobs=-1)
        model.fit(trainX_pca, y_train)

        y_pred = model.predict_proba(validX_pca)[:, 1]
        print(y_valid.shape)
        print(y_pred.shape)
        current_loss = roc_auc_score(y_valid, y_pred)
        train_loss = roc_auc_score(y_valid, y_pred)
        print(f"k-f val AUC {current_loss}")
        print(f"train AUC {train_loss}")
        oof_preds[val_] = y_pred

        X_test_pca = pca.transform(X_test.reshape(len(X_test), -1, ))
        test_pred = model.predict_proba(X_test_pca)[:, 1]
        if preds is None:
            preds = test_pred
        else:
            preds += test_pred
        del model

    cv_loss = roc_auc_score(y, oof_preds)
    print('ALL FOLDS AUC: %.5f ' % cv_loss)
    oof_preds_df = pd.DataFrame()
    oof_preds_df['EventID'] = np.unique(data_train['EventID'].values)
    oof_preds_df['Proba'] = oof_preds
    oof_preds_df.to_csv('oof_preds.csv', index=False)
    return cv_loss, preds / folds.n_splits

In [30]:
best_loss, preds = do_train()
print('CV:', best_loss)

train len 50000
valid len 10000


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 500building tree 2 of 500

building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.3s


building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69 of 500
building tree 70 of 500
building tree 71 of 500
building tree 72

...

[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   23.3s


building tree 127 of 500
building tree 128 of 500
building tree 129 of 500
building tree 130 of 500
building tree 131 of 500
building tree 132 of 500
building tree 133 of 500
building tree 134 of 500
building tree 135 of 500
building tree 136 of 500
building tree 137 of 500
building tree 138 of 500
building tree 139 of 500
building tree 140 of 500
building tree 141 of 500
building tree 142 of 500
building tree 143 of 500
building tree 144 of 500
building tree 145 of 500
building tree 146 of 500
building tree 147 of 500
building tree 148 of 500
building tree 149 of 500
building tree 150 of 500
building tree 151 of 500
building tree 152 of 500
building tree 153 of 500
building tree 154 of 500
building tree 155 of 500
building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500
building tree 166 of 500


[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:   53.3s


building tree 287 of 500
building tree 288 of 500
building tree 289 of 500
building tree 290 of 500
building tree 291 of 500
building tree 292 of 500
building tree 293 of 500
building tree 294 of 500
building tree 295 of 500
building tree 296 of 500
building tree 297 of 500
building tree 298 of 500
building tree 299 of 500
building tree 300 of 500
building tree 301 of 500
building tree 302 of 500
building tree 303 of 500
building tree 304 of 500
building tree 305 of 500
building tree 306 of 500
building tree 307 of 500
building tree 308 of 500
building tree 309 of 500
building tree 310 of 500
building tree 311 of 500
building tree 312 of 500
building tree 313 of 500
building tree 314 of 500
building tree 315 of 500
building tree 316 of 500
building tree 317 of 500
building tree 318 of 500
building tree 319 of 500
building tree 320 of 500


KeyboardInterrupt: ignored

In [0]:
submission = pd.DataFrame()
submission['EventID'] = np.unique(data_test['EventID'].values)
submission['Proba'] = preds
submission.to_csv(f's_pca{pca_num}.csv', index=False, float_format='%.6f')